# Setup

In [1]:
from langchain_groq import ChatGroq
from langchain.cache import InMemoryCache,SQLiteCache
from langchain.globals import set_llm_cache
import os
import json
import hashlib


In [2]:
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

True

In [3]:
groq = ChatGroq(model_name='llama-3.1-8b-instant')
set_llm_cache(InMemoryCache())

# Memória

In [4]:
prompt = 'Me diga em poucas palavras que foi Carl Sagan.'
response1 = groq.invoke(prompt)
print("Primeira resposta (API chamada):", response1)

Primeira resposta (API chamada): content='Carl Sagan foi um astrônomo e cosmólogo americano, conhecido por seu livro "O Pântano de Júpiter" e a série de TV "Cosmos".' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 49, 'total_tokens': 90, 'completion_time': 0.066606967, 'prompt_time': 0.009274622, 'queue_time': 0.273348746, 'total_time': 0.075881589}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_46fc01befd', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--b259ef51-9a0e-450d-bfaf-58d0d44d812a-0' usage_metadata={'input_tokens': 49, 'output_tokens': 41, 'total_tokens': 90}


In [5]:
response2 = groq.invoke(prompt)
print("Segunda resposta (usando cache):", response2)

Segunda resposta (usando cache): content='Carl Sagan foi um astrônomo e cosmólogo americano, conhecido por seu livro "O Pântano de Júpiter" e a série de TV "Cosmos".' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 49, 'total_tokens': 90, 'completion_time': 0.066606967, 'prompt_time': 0.009274622, 'queue_time': 0.273348746, 'total_time': 0.075881589}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_46fc01befd', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--b259ef51-9a0e-450d-bfaf-58d0d44d812a-0' usage_metadata={'input_tokens': 49, 'output_tokens': 41, 'total_tokens': 90, 'total_cost': 0}


# Disco / Banco de Dados

In [6]:
set_llm_cache(SQLiteCache(database_path="groq_cache.db"))

In [7]:
prompt = 'Me diga em poucas palavras quem foi Neil Armstrong.'

In [8]:
response1 = groq.invoke(prompt)
print("Primeira resposta (API chamada):", response1)

Primeira resposta (API chamada): content='Neil Armstrong foi um astronauta norte-americano que foi o primeiro homem a pisar na Lua em 20 de julho de 1969, durante a Missão Apolo 11.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 48, 'total_tokens': 90, 'completion_time': 0.048800924, 'prompt_time': 0.006565854, 'queue_time': 0.68662994, 'total_time': 0.055366778}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--e7f8fa37-79db-415f-98d5-88f7747f46ac-0' usage_metadata={'input_tokens': 48, 'output_tokens': 42, 'total_tokens': 90}


In [9]:
response2 = groq.invoke(prompt)
print("Segunda resposta (usando cache):", response2)

Segunda resposta (usando cache): content='Neil Armstrong foi um astronauta norte-americano que foi o primeiro homem a pisar na Lua em 20 de julho de 1969, durante a Missão Apolo 11.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 48, 'total_tokens': 90, 'completion_time': 0.048800924, 'prompt_time': 0.006565854, 'queue_time': 0.68662994, 'total_time': 0.055366778}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--e7f8fa37-79db-415f-98d5-88f7747f46ac-0' usage_metadata={'input_tokens': 48, 'output_tokens': 42, 'total_tokens': 90, 'total_cost': 0}


# Personalizado

In [10]:
class SimpleDiskCache:
    def __init__(self, cache_dir='cache_dir'):
        self.cache_dir = cache_dir
        os.makedirs(self.cache_dir, exist_ok=True)

    def _get_cache_path(self, key):
        hashed_key = hashlib.md5(key.encode()).hexdigest() #hasg cria nome de arquivo único
        return os.path.join(self.cache_dir, f"{hashed_key}.json")

    def lookup(self, key, llm_string):
        cache_path = self._get_cache_path(key)
        if os.path.exists(cache_path):
            with open(cache_path, 'r') as f:
                return json.load(f)
        return None

    def update(self, key, value, llm_string):
        cache_path = self._get_cache_path(key)
        with open(cache_path, 'w') as f:
            #json.dump(value.to_json(), f)
            print(value)

In [11]:
cache = SimpleDiskCache()
set_llm_cache(cache)
prompt = 'Me diga em poucas palavras quem foi Neil Degrasse Tyson.'

In [12]:
def invoke_with_cache(llm, prompt, cache):
    cached_response = cache.lookup(prompt, "")
    if cached_response:
        print("Usando cache:")
        return cached_response

    response = llm.invoke(prompt)
    cache.update(prompt, response, "")
    return response

In [ ]:
response1 = invoke_with_cache(groq, prompt, cache)
response_text1 = response1.replace('\n', ' ') 

print("Primeira resposta (API chamada):", response_text1)

In [ ]:
response2 = invoke_with_cache(groq, prompt, cache)
response_text2 = response2.replace('\n', ' ')  

print("Segunda resposta (usando cache):", response_text2)